In [1]:
from PIL import Image, ImageEnhance
import numpy as np 
import os
import cv2
from skimage.color import rgb2lab, lab2rgb
from skimage import color
from skimage.io import imsave
# np.save('example_1', a)
# b = np.load('example/example_1.npy')
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, ReLU
from keras.layers import Activation, Dense, Dropout, Flatten
#from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import load_model
import pandas as pd 
# import tensorflow as tf
# from keras.models import load_model
# import keras 

In [2]:
# image_folder = "/Users/alexeyvaganov/doc/files/color_matcher/type"
image_massiv = "/Users/alexeyvaganov/doc/files/color_matcher/temp"

In [3]:
pixel = 1

# Обучение

In [4]:
def show_image(img_file):
    img = Image.open(img_file)
    img.show()
# array_transform_nastr = [
#     [1,1,0.8],
#     [1,1,2]
# ]
array_transform_nastr = [
    [1.5,0.5,0.5],
    [1.5,1,0.5],
    [1,1,0.8],
    [1,1,2],
    [2,1.2,0],
    [2,1.2,0.1]
]

In [5]:
def transform_image(img_pil,brightness,contrast,color):
    if (brightness != 0):
        enhancer = ImageEnhance.Brightness(img_pil)
        img_pil = enhancer.enhance(brightness)
    if (contrast != 0):
        enhancer = ImageEnhance.Contrast(img_pil)
        img_pil = enhancer.enhance(contrast)
    if (color != 0):
        enhancer = ImageEnhance.Color(img_pil)
        img_pil = enhancer.enhance(color) 
    return img_pil

In [11]:
def crop_image(array_x_y,a,b):
    h=0
    while h <= a.shape[0]-pixel:
        w=0
        while w <= a.shape[1]-pixel:
            #print (h,",",h+56)
            array_x_y.append([a[h,w][1],a[h,w][2],b[h,w][1],b[h,w][2]])
#             x_ = a[h,w]
#             y_ = b[h,w]
#             array_x_y.append([x_,y_])
            w+=pixel
        h+=pixel

# Из одной папки

In [7]:
image_folder = "/Users/alexeyvaganov/doc/files/color_matcher/slide_X"
array_image = []
for root, dirs, files in os.walk(image_folder):
    for file in files:
        if file == ".DS_Store":
            continue
        img_pil = Image.open(root + "/" + file)
        img_pil = img_pil.convert('RGB')
        img_pil_lab = rgb2lab(img_pil)
        #img_cv2_ish = cv2.imread(root + "/" + file)
        for x in array_transform_nastr:
            img_ = transform_image(img_pil,x[0],x[1],x[2])
            #img_.save("/Users/alexeyvaganov/doc/files/color_matcher/temp/geeks.jpg")
            #img_cv2_transf = cv2.imread("/Users/alexeyvaganov/doc/files/color_matcher/temp/geeks.jpg")
            img_lab = rgb2lab(img_)
            #img_pil = rgb2lab(img_pil)
            array_image.append([np.array(img_lab),np.array(img_pil_lab)])
        
array_full_image = np.array(array_image) 
array_full_image.shape

/var/folders/87/c02w9bp14k1fx5943h90k0500000gn/T/ipykernel_42487/3218737492.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array_full_image = np.array(array_image)


(30, 2)

In [56]:
imsave("/Users/alexeyvaganov/doc/files/color_matcher/temp/tttt.jpg", lab2rgb(array_full_image[0,1]))


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


# Подготовка данных

In [12]:
data = []
i=0
x_sgape = array_full_image.shape[0]
while i < x_sgape:
    crop_image(data,array_full_image[i,0],array_full_image[i,1])
    i+=1

dFrame = pd.DataFrame(data) 

# array_x_y = []
# i=0
# x_sgape = array_full_image.shape[0]
# while i < x_sgape:
#     crop_image(array_x_y,array_full_image[i,0],array_full_image[i,1])
#     i+=1
# array_crop_image = np.array(array_x_y) 
# array_crop_image = 1.0/255*array_crop_image
# array_crop_image.shape

In [13]:
dFrame

,0,1,2,3
0,-0.001059,0.002007,0.000000,0.000000
1,-0.001059,0.002007,0.000000,0.000000
2,-0.001059,0.002007,0.000000,0.000000
3,-0.001059,0.002007,0.000000,0.000000
4,-0.001059,0.002007,0.000000,0.000000
...,...,...,...,...
22490203,-0.292302,-2.467918,-0.673269,-11.512144
22490204,-0.504999,-2.011924,-2.066120,-9.529143
22490205,-0.504999,-2.011924,-2.066120,-9.529143
22490206,-0.291185,-2.450675,-0.619631,-10.782802


In [14]:
dFrame_ = dFrame.drop_duplicates()

In [15]:
dFrame_

,0,1,2,3
0,-0.001059,0.002007,0.000000,0.000000
144,-0.215369,0.588665,-0.139061,0.378457
152,-0.215369,0.588665,0.122438,0.470595
496,0.620613,-0.441352,0.662042,-0.194147
872,-0.001059,0.002007,0.261499,0.092139
...,...,...,...,...
22490199,-0.703658,-1.453317,-3.093772,-6.429565
22490200,-0.699564,-1.443880,-3.091510,-6.405489
22490202,-0.297403,-2.555699,-0.854491,-11.174781
22490203,-0.292302,-2.467918,-0.673269,-11.512144


In [16]:
array_crop_image = np.array(dFrame_) 
array_crop_image = 1.0/255*array_crop_image
array_crop_image.shape

(2472496, 4)

In [66]:
X_ = array_crop_image[:,0:2]
Y_ = array_crop_image[:,2:4]
# X_ = X_[:,1:]
# Y_ = Y_[:,1:]

# KERAS

In [67]:
# модель 1

model = Sequential()
model.add(Dense(100, input_dim=2, activation='sigmoid'))
model.add(ReLU(1.0))
model.add(Dense(50, activation='sigmoid'))
model.add(ReLU(1.0))
model.add(Dense(25, activation='softmax'))

#IMPORTANT PART
model.add(Dense(2, activation='linear'))

model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])


In [72]:
# модель 2

model = Sequential()
model.add(Dense(20, input_dim=2, activation='sigmoid'))
model.add(ReLU(1.0))
model.add(Dense(10, activation='sigmoid'))
model.add(ReLU(1.0))
model.add(Dense(5, activation='softmax'))

#IMPORTANT PART
model.add(Dense(2, activation='linear'))

model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

In [74]:
model.fit(X_,Y_,batch_size=256, epochs=25, validation_split=0.2)

Epoch 1/25
7727/7727 [==============================] - 47s 6ms/step - loss: 0.0015 - accuracy: 0.9213 - val_loss: 0.0027 - val_accuracy: 0.9226
Epoch 2/25
7727/7727 [==============================] - 47s 6ms/step - loss: 0.0015 - accuracy: 0.9227 - val_loss: 0.0027 - val_accuracy: 0.9195
Epoch 3/25
7727/7727 [==============================] - 48s 6ms/step - loss: 0.0015 - accuracy: 0.9238 - val_loss: 0.0026 - val_accuracy: 0.9252
Epoch 4/25
7727/7727 [==============================] - 48s 6ms/step - loss: 0.0015 - accuracy: 0.9250 - val_loss: 0.0027 - val_accuracy: 0.9252
Epoch 5/25
7727/7727 [==============================] - 48s 6ms/step - loss: 0.0015 - accuracy: 0.9260 - val_loss: 0.0027 - val_accuracy: 0.9260
Epoch 6/25
7727/7727 [==============================] - 48s 6ms/step - loss: 0.0015 - accuracy: 0.9268 - val_loss: 0.0027 - val_accuracy: 0.9236
Epoch 7/25
7727/7727 [==============================] - 49s 6ms/step - loss: 0.0015 - accuracy: 0.9271 - val_loss: 0.0026 - val_ac

In [27]:
model.save("/Users/alexeyvaganov/doc/files/color_matcher/temp/temp_keras_regression.h5")

# Предсказание

In [42]:
Obr = 90

In [48]:
file_name_massiv = []
#file_name_massiv.append("IMG_2837_m.jpeg")
file_name_massiv.append("IMG_2825.jpeg")
file_name_massiv.append("IMG_3379.jpeg")
file_name_massiv.append("IMG_2915.jpeg")
file_name_massiv.append("IMG_2980.jpeg")
file_name_massiv.append("IMG_3107.jpeg")
file_name_massiv.append("IMG_3323.jpeg")
file_name_massiv.append("IMG_3336.jpeg")
file_name_massiv.append("IMG_2825.jpeg")
file_name_massiv.append("IMG_2769.jpeg")
file_name_massiv.append("IMG_2837.jpeg")
file_name_massiv.append("IMG_2847.jpeg")

In [49]:
def add_testing_file(file_name):
    test_image_path = image_massiv + "/" + file_name
    img_test = Image.open(test_image_path)
    img_test = img_test.convert('RGB')
    img_test = rgb2lab(img_test)
    return np.array(img_test) 

In [50]:
arrive_testing_file = []
for name_test_file in file_name_massiv:
    arr_ = add_testing_file(name_test_file)
    arrive_testing_file.append([name_test_file, arr_])

In [51]:
def predict_image(array_full_img_test):
    new_img = np.zeros((array_full_img_test.shape[0], array_full_img_test.shape[1], 3))
    data_p = []
    h=0
    while h <= array_full_img_test.shape[0]-pixel:
        w=0
        while w <= array_full_img_test.shape[1]-pixel:
            data_p.append([array_full_img_test[h,w][1],array_full_img_test[h,w][2]])
            w+=pixel
        h+=pixel 
        
    x_ = np.array(data_p)
    x_ = 1.0/255*x_
    #x_ = x_.reshape(1,2)
    pred_y = model.predict(x_)
    
    h=0
    i=0
    while h <= array_full_img_test.shape[0]-pixel:
        w=0
        while w <= array_full_img_test.shape[1]-pixel:
            new_img[h,w] = [array_full_img_test[h,w][0],pred_y[i][0]*256,pred_y[i][1]*256]
            i+=1
            w+=pixel
        h+=pixel 

    return new_img

# Предсказание здесь

In [75]:
Obr += 1
model_name = image_massiv + "/" + str(Obr) + "_type" + ".h5"
print(model_name)
for m_image in arrive_testing_file:
    p_image = predict_image(m_image[1])
    image_name = image_massiv + "/" + str(Obr) + "_" + m_image[0]
    print(image_name)
    imsave(image_name, lab2rgb(p_image))
model.save(model_name)

/Users/alexeyvaganov/doc/files/color_matcher/temp/95_type.h5
   84/38400 [..............................] - ETA: 1:10

2022-06-06 21:53:15.251193: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


38400/38400 [==============================] - 64s 2ms/step


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


/Users/alexeyvaganov/doc/files/color_matcher/temp/95_IMG_2825.jpeg
38400/38400 [==============================] - 61s 2ms/step


...

/Users/alexeyvaganov/doc/files/color_matcher/temp/95_IMG_2769.jpeg
38400/38400 [==============================] - 62s 2ms/step


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


/Users/alexeyvaganov/doc/files/color_matcher/temp/95_IMG_2837.jpeg
38400/38400 [==============================] - 62s 2ms/step


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


/Users/alexeyvaganov/doc/files/color_matcher/temp/95_IMG_2847.jpeg


# Старые

In [124]:
array_full_img_test = arrive_testing_file[0][1]
new_img = np.zeros((array_full_img_test.shape[0], array_full_img_test.shape[1], 3))
h=0
while h <= array_full_img_test.shape[0]-pixel:
    w=0
    while w <= array_full_img_test.shape[1]-pixel:
        new_img[h,w] = [array_full_img_test[h,w][0],array_full_img_test[h,w][1],array_full_img_test[h,w][2]]
        w+=pixel
    h+=pixel 
imsave("/Users/alexeyvaganov/doc/files/color_matcher/temp/82_IMG_2837_m22.jpeg", lab2rgb(new_img))
#(color.convert_colorspace(new_img, 'HSV', 'RGB')*255).astype(np.uint8)
#(color.convert_colorspace(lab2rgb(new_img), 'HSV', 'RGB')*255).astype(np.uint8)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [81]:
file_name = "IMG_2825.jpeg"
test_image_path = image_massiv + "/" + file_name
img_test = Image.open(test_image_path)
img_test = img_test.convert('RGB')
#array_img_test.append(img_test) 
img_test = rgb2lab(img_test)
array_full_img_test = np.array(img_test) 
array_full_img_test.shape

(1280, 960, 3)

In [46]:
model.save(image_massiv + "/" + str(Obr) + "type_.h5")

In [24]:
model = load_model("/Users/alexeyvaganov/doc/files/color_matcher/temp/temp_keras_regression.h5")